In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
# pd.options.display.max_rows = 999
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from pprint import pprint

In this exercise, we will <b>build a logistic regression model from scratch</b> and use it to <b>classify whether a patient has diabetes or not</b>.

# Load iris dataset
- Load the iris dataset from sklearn, use data for the classes 0 and 1 only for this example
- Split chosen data into train set and test set

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
## testing purpose
# X = np.append(iris.data[:3], iris.data[50:53], axis=0)
# y = np.append(iris.target[:3], iris.target[50:53], axis=0)[:, np.newaxis]
X = iris.data[:100]
y = iris.target[:100][:, np.newaxis]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y, random_state=42)

# Construct logistic regression model

## Compute prediction

### Write sigmoid function
- Recall sigmoid function is defined by:<br> 
$$S(x) = \frac{1}{1 + e^{-x}}$$

In [3]:
# sigmoid function
def sigmoid(x):
    """
    Args:
        x (numpy array): the linear combination of features
    
    Returns:
        return a probability (0-1)
    """
    return 1 / (1 + np.exp(-x))

In [4]:
# test sigmoid
sigmoid(0)

0.5

### Write predict function
- The predicted probabiliy of logistic regression model is given by:<br>
$$h_{\beta}(x) = S(\beta \cdot x)$$

In [5]:
# y_hat function
def y_hat(beta, X):
    """
    Args:
        beta (numpy array): weight vector in linear regression, dimension 1 x n
        X (numpy array): training data, dimension m x n
        
    Returns:
        return predicted probabilities, dimension 1 x n
    """
    return sigmoid(np.dot(X, beta))

In [6]:
# choose the first two rows of training data for trial
beta = np.zeros((X.shape[1], 1))
Xtry = Xtrain[0:2]
ytry = ytrain[0:2]

In [7]:
# test yhat with trial data
yhat = y_hat(beta, Xtry)
yhat

array([[0.5],
       [0.5]])

## Write cost function
- Suppose we have $m$ rows of training data and $n$ features
- Recall the cost function of logistic regression is given by:<br> 
$$J(\beta) = -\frac{1}{m}\sum_{i=1}^{m}[y_i log(\hat{y_i})+(1-y_i)log(1-\hat{y_i})]$$
where $y_i$ and $\hat{y_i}$ are the true values and predicted values of i-th row of training data respectively.

In [8]:
# cost function per row of data
def cost_per_row(y, yhat):
    if y == 1:
        return np.multiply(y, np.log(yhat)) 
    else:
        return np.multiply(1-y, np.log(1 - yhat))

In [9]:
# test cost per row function
cost_per_row(0, 0.5)

-0.6931471805599453

In [10]:
# cost function of logistic regression
def cost(Y, Yhat):
    """
    Args:
        y (numpy array): labels of training data, dimension 1 x m
        yhat (numpy array): predictied values of training data, dimension 1 x m
        
    Returns:
        return the cost of beta
    """
    m = len(y)
    df1 = pd.DataFrame(data={'y': Y.ravel(), 'yhat': Yhat.ravel()})
    diff = df1.apply(lambda row: cost_per_row(row['y'], row['yhat']), axis=1).to_numpy().reshape(-1, 1)
    return -np.mean(diff)

In [11]:
# test cost functon
cost(ytry, yhat)

0.6931471805599453

## Calculate the gradient
- We're going to use gradient descent for optimizing the cost function above.
- Write a function to compute gradient for updating the weight $\beta$ when feeding each row of training data
- Recall in sec 2.1 the weight updating formula is given by:<br>
$$\beta_{k+1} = \beta_{k} - \alpha \frac{\partial J}{\partial \beta}(\beta_k)$$
where $\alpha$ is a fixed learning rate, $k$ is the number of iterations/epochs.
- The gradient is<br><br>
$$\frac{\partial J}{\partial \beta} = (\frac{\partial J}{\partial \beta_0}, \frac{\partial J}{\partial \beta_1},..., \frac{\partial J}{\partial \beta_n})$$
- We only need to compute $$\frac{\partial J}{\partial \beta_j} = \frac{1}{m}\sum_{i=1}^{m}(\hat{y_i}-y_i)x_{ij}$$ for each $j = 0, 1, 2,... ,n$.
- Derivation of the gradient formula of logistic regression at p.18 of:<br> 
https://see.stanford.edu/materials/aimlcs229/cs229-notes1.pdf

In [12]:
# function to compute gradient
def gradient(beta, X, y, yhat):
    # initialize the gradient as a zero vector
    grad = np.zeros(beta.shape)
    
    # compute delta J/delta beta for each j
    for j in range(X.shape[1]):
        first = np.multiply(yhat - y, X[:, j][: ,np.newaxis])
        grad[j] = np.mean(first)
    
    return grad

In [13]:
# test gradient function
gradient(beta, Xtry, ytry, yhat)

array([[-3.  ],
       [-1.45],
       [-2.05],
       [-0.7 ]])

## Write function for gradient descent
- Initialize beta as a zero vector, for each epoch, feed the whole train set to calculate: 
    1. yhat (prediction of training data using existing beta)
    2. gradient vector
    3. update beta using formula in 2.3
    4. cost/loss of existing prediction
- Remember to add a constant column (e.g. a column with all 1) to Xtrain with represents the <b>intercept</b> of linear regression

In [14]:
def train_grad_desecnt(X, y, epochs=1000, alpha=0.1):
    """
    Args:
        epochs (int): no. of iterations allowed for optimization
        alpha (floats): learning rate
    
    Returns:
        beta (numpy array): the final weight
        loss (float): the final cost
    """
    m, n = X.shape
    beta = np.zeros((n+1, 1))    # +1 accounts for constant coefficient beta_0
    X = np.hstack((np.ones((m, 1)), X))    # add a constant column for constant coefficient
    losses = []
    
    # training loop
    for epoch in range(epochs):
        yhat = y_hat(beta, X)
        grad = gradient(beta, X, y, yhat)
                        
        # update beta and loss
        beta -= alpha*grad
        loss = cost(y, yhat)
        losses.append(loss)
        
        # log result of each epoch
        print("*"*50)
        print(f'epoch = {epoch}')
        print(f'beta = {beta}')
        print(f'grad = {grad}')
        print(f'yhat = {yhat[:50]}')
        print(f'y = {y[:50]}')
        print(f'loss = {loss}')
        
    return beta, losses

In [15]:
%%time
# fit training set into train_gard_descent function
beta_final, losses = train_grad_desecnt(Xtrain, ytrain)
print('beta_final = ', beta_final)
print('loss = ', losses[-1])

**************************************************
epoch = 0
beta = [[ 0.00066667]
 [ 0.0286    ]
 [-0.0156    ]
 [ 0.0714    ]
 [ 0.02706667]]
grad = [[-0.00666667]
 [-0.286     ]
 [ 0.156     ]
 [-0.714     ]
 [-0.27066667]]
yhat = [[0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]]
loss = 0.6931471805599454
**************************************************
epoch = 1
beta = [[-0.00697957]
 [ 0.00963837]
 [-0.05585879]
 [ 0.11402806]
 [ 0.0

**************************************************
epoch = 49
beta = [[-0.13542967]
 [-0.17453374]
 [-0.80992934]
 [ 1.17896457]
 [ 0.49999363]]
grad = [[ 0.01378923]
 [ 0.01894411]
 [ 0.07985542]
 [-0.11406315]
 [-0.04953872]]
yhat = [[0.88971665]
 [0.86774791]
 [0.0631332 ]
 [0.88630603]
 [0.88765443]
 [0.10426998]
 [0.91797999]
 [0.06191132]
 [0.16580374]
 [0.89576314]
 [0.8941642 ]
 [0.10285605]
 [0.88532655]
 [0.94139309]
 [0.87793651]
 [0.09333849]
 [0.88589803]
 [0.13663095]
 [0.1617152 ]
 [0.18517921]
 [0.10548499]
 [0.90300859]
 [0.1032107 ]
 [0.92207226]
 [0.17002396]
 [0.90250368]
 [0.12472649]
 [0.92842097]
 [0.79872504]
 [0.9378861 ]
 [0.90730946]
 [0.87871939]
 [0.12044391]
 [0.15650508]
 [0.11584799]
 [0.11805084]
 [0.12897272]
 [0.87802576]
 [0.9395676 ]
 [0.14668738]
 [0.17309173]
 [0.80538441]
 [0.92080415]
 [0.85897143]
 [0.19216901]
 [0.87883261]
 [0.93160942]
 [0.10960359]
 [0.13334432]
 [0.90651726]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]


**************************************************
epoch = 97
beta = [[-0.18294678]
 [-0.2390532 ]
 [-1.08459352]
 [ 1.57216781]
 [ 0.67235137]]
grad = [[ 0.00741023]
 [ 0.00995944]
 [ 0.04281456]
 [-0.0613915 ]
 [-0.02716291]]
yhat = [[0.9401441 ]
 [0.92239999]
 [0.02470804]
 [0.93713622]
 [0.93861484]
 [0.05043836]
 [0.960379  ]
 [0.02406907]
 [0.09843961]
 [0.94446973]
 [0.94339475]
 [0.04935536]
 [0.93678591]
 [0.97536248]
 [0.93106967]
 [0.0429899 ]
 [0.93750027]
 [0.07420104]
 [0.09464662]
 [0.11565096]
 [0.05132655]
 [0.95009846]
 [0.0496633 ]
 [0.96324132]
 [0.10224859]
 [0.94990018]
 [0.06554705]
 [0.96750883]
 [0.85848845]
 [0.97325679]
 [0.95327736]
 [0.93198695]
 [0.06190643]
 [0.09059322]
 [0.0586043 ]
 [0.06003983]
 [0.06842396]
 [0.93091396]
 [0.97413244]
 [0.08257222]
 [0.10496212]
 [0.86529664]
 [0.96226241]
 [0.91526396]
 [0.12197538]
 [0.93153889]
 [0.96934226]
 [0.05406889]
 [0.07195766]
 [0.95222135]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]


 [0.96845913]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]]
loss = 0.0475784491002116
**************************************************
epoch = 144
beta = [[-0.21146691]
 [-0.27717242]
 [-1.24956161]
 [ 1.80884304]
 [ 0.77773894]]
grad = [[ 0.00506638]
 [ 0.00673726]
 [ 0.02935218]
 [-0.04212621]
 [-0.01887607]]
yhat = [[0.95934165]
 [0.94474584]
 [0.01399622]
 [0.95683175]
 [0.95811479]
 [0.03230622]
 [0.97488461]
 [0.01357468]
 [0.0712113 ]
 [0.9627031 ]
 [0.96187627]
 [0.03146816]
 [0.95663055]
 [0.98561743]
 [0.95200282]
 [0.02676044]
 [0.95728724]
 [0.05091713]
 [0.06791636]
 [0.08626498]
 [0.03299425]
 [0.96713223]
 [0.03171972]
 [0.97702286]
 [0.07455103]
 [0.96703727]
 [0.04406158]
 [0.9801298 ]
 [0.88733726]
 [0.98415357]
 [0.96961088]
 [0.9528153 ]
 [0.04109798]
 [0.06

**************************************************
epoch = 190
beta = [[-0.23168918]
 [-0.30397043]
 [-1.36693625]
 [ 1.97731386]
 [ 0.85359793]]
grad = [[ 0.00385974]
 [ 0.00509829]
 [ 0.02244835]
 [-0.03222117]
 [-0.01457775]]
yhat = [[0.96933834]
 [0.9569175 ]
 [0.00934359]
 [0.9671997 ]
 [0.96830517]
 [0.02349164]
 [0.98195817]
 [0.00903222]
 [0.05639975]
 [0.97209866]
 [0.97141891]
 [0.022811  ]
 [0.96705331]
 [0.99025067]
 [0.96314834]
 [0.01907703]
 [0.96764162]
 [0.03886773]
 [0.05349696]
 [0.06981837]
 [0.02405496]
 [0.97573413]
 [0.02302231]
 [0.98365465]
 [0.05937798]
 [0.9756859 ]
 [0.03313262]
 [0.98607488]
 [0.90484314]
 [0.98914152]
 [0.97776536]
 [0.96385744]
 [0.03064526]
 [0.05054395]
 [0.02857264]
 [0.02944126]
 [0.03496113]
 [0.96298272]
 [0.98957084]
 [0.04478436]
 [0.06144496]
 [0.9110258 ]
 [0.98306693]
 [0.95169381]
 [0.07490293]
 [0.96338075]
 [0.98705606]
 [0.02568557]
 [0.03740594]
 [0.97699315]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]

...

**************************************************
epoch = 793
beta = [[-0.33634333]
 [-0.44021912]
 [-1.98755798]
 [ 2.86631817]
 [ 1.26749705]]
grad = [[ 0.00091929]
 [ 0.00118191]
 [ 0.00559668]
 [-0.00798815]
 [-0.00383365]]
yhat = [[0.99358072]
 [0.98927996]
 [0.00113216]
 [0.99287943]
 [0.99322323]
 [0.00438662]
 [0.99705713]
 [0.00107211]
 [0.01629065]
 [0.99440534]
 [0.99418689]
 [0.00419784]
 [0.9928081 ]
 [0.99882353]
 [0.99148349]
 [0.00322786]
 [0.99303462]
 [0.00934554]
 [0.01506758]
 [0.02252874]
 [0.00455964]
 [0.99541759]
 [0.00426539]
 [0.99746111]
 [0.01767329]
 [0.99543619]
 [0.00731201]
 [0.99798641]
 [0.96392485]
 [0.99860929]
 [0.99599707]
 [0.99172066]
 [0.00651204]
 [0.01377143]
 [0.00589619]
 [0.00615778]
 [0.00793346]
 [0.99142163]
 [0.99868935]
 [0.01149865]
 [0.01854732]
 [0.96770471]
 [0.99733262]
 [0.9871811 ]
 [0.02509926]
 [0.99152405]
 [0.99820608]
 [0.00500099]
 [0.00879578]
 [0.99579199]]
y = [[1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]

**************************************************
epoch = 841
beta = [[-0.34062296]
 [-0.44571891]
 [-2.01364912]
 [ 2.90354998]
 [ 1.28539007]]
grad = [[ 0.00086604]
 [ 0.00111249]
 [ 0.00528707]
 [-0.00754301]
 [-0.00362989]]
yhat = [[9.94001920e-01]
 [9.89911683e-01]
 [1.03737336e-03]
 [9.93337383e-01]
 [9.93661537e-01]
 [4.09090909e-03]
 [9.97278371e-01]
 [9.81384033e-04]
 [1.54665503e-02]
 [9.94782111e-01]
 [9.94574493e-01]
 [3.91271441e-03]
 [9.93267419e-01]
 [9.98925539e-01]
 [9.92007966e-01]
 [2.99844018e-03]
 [9.93483110e-01]
 [8.80821912e-03]
 [1.42918573e-02]
 [2.14856982e-02]
 [4.25518158e-03]
 [9.95735525e-01]
 [3.97680689e-03]
 [9.97656571e-01]
 [1.68006838e-02]
 [9.95754066e-01]
 [6.86498637e-03]
 [9.98146533e-01]
 [9.65450404e-01]
 [9.98726605e-01]
 [9.96282253e-01]
 [9.92232403e-01]
 [6.10563577e-03]
 [1.30427063e-02]
 [5.52200701e-03]
 [5.77041683e-03]
 [7.45821769e-03]
 [9.91949301e-01]
 [9.98800981e-01]
 [1.08637822e-02]
 [1.76397030e-02]
 [9.69129919e-01]
 [9.9753

**************************************************
epoch = 879
beta = [[-0.34383873]
 [-0.44984842]
 [-2.03330217]
 [ 2.93158405]
 [ 1.29889501]]
grad = [[ 0.00082803]
 [ 0.00106298]
 [ 0.00506569]
 [-0.00722477]
 [-0.0034839 ]]
yhat = [[9.94301386e-01]
 [9.90363808e-01]
 [9.71326926e-04]
 [9.93663435e-01]
 [9.93973412e-01]
 [3.88161359e-03]
 [9.97434192e-01]
 [9.18212027e-04]
 [1.48739682e-02]
 [9.95049557e-01]
 [9.94849774e-01]
 [3.71101769e-03]
 [9.93594508e-01]
 [9.98996582e-01]
 [9.92382257e-01]
 [2.83664432e-03]
 [9.93802314e-01]
 [8.42444609e-03]
 [1.37345851e-02]
 [2.07326156e-02]
 [4.03957873e-03]
 [9.95960690e-01]
 [3.77261293e-03]
 [9.97793984e-01]
 [1.61725570e-02]
 [9.95979139e-01]
 [6.54666416e-03]
 [9.98258805e-01]
 [9.66560548e-01]
 [9.98808488e-01]
 [9.96483861e-01]
 [9.92597477e-01]
 [5.81664580e-03]
 [1.25198125e-02]
 [5.25621789e-03]
 [5.49512916e-03]
 [7.11947403e-03]
 [9.92325909e-01]
 [9.98878841e-01]
 [1.04092145e-02]
 [1.69859920e-02]
 [9.70165268e-01]
 [9.9768

**************************************************
epoch = 926
beta = [[-0.34762626]
 [-0.4547088 ]
 [-2.05650337]
 [ 2.96466728]
 [ 1.31486815]]
grad = [[ 0.00078534]
 [ 0.00100743]
 [ 0.00481673]
 [-0.00686696]
 [-0.00331944]]
yhat = [[9.94636296e-01]
 [9.90872570e-01]
 [8.98824311e-04]
 [9.94028551e-01]
 [9.94322458e-01]
 [3.64849250e-03]
 [9.97606934e-01]
 [8.48914260e-04]
 [1.42041329e-02]
 [9.95348204e-01]
 [9.95157322e-01]
 [3.48647132e-03]
 [9.93960852e-01]
 [9.99074516e-01]
 [9.92802314e-01]
 [2.65702945e-03]
 [9.94159648e-01]
 [7.99335187e-03]
 [1.31052090e-02]
 [1.98781167e-02]
 [3.79931359e-03]
 [9.96211596e-01]
 [3.54523424e-03]
 [9.97946060e-01]
 [1.54618371e-02]
 [9.96229887e-01]
 [6.19009701e-03]
 [9.98382750e-01]
 [9.67829256e-01]
 [9.98898503e-01]
 [9.96708136e-01]
 [9.93007061e-01]
 [5.49335594e-03]
 [1.19299089e-02]
 [4.95917820e-03]
 [5.18733503e-03]
 [6.73968537e-03]
 [9.92748602e-01]
 [9.98964375e-01]
 [9.89742286e-03]
 [1.62459237e-02]
 [9.71346605e-01]
 [9.9783

beta = [[-0.35137244]
 [-0.45951263]
 [-2.07950989]
 [ 2.99745969]
 [ 1.33073944]]
grad = [[ 0.00074523]
 [ 0.00095529]
 [ 0.00458245]
 [-0.0065303 ]
 [-0.00316438]]
yhat = [[9.94949520e-01]
 [9.91351555e-01]
 [8.32351148e-04]
 [9.94370498e-01]
 [9.94649164e-01]
 [3.43138018e-03]
 [9.97766969e-01]
 [7.85428829e-04]
 [1.35703166e-02]
 [9.95627054e-01]
 [9.95444639e-01]
 [3.27745556e-03]
 [9.94304023e-01]
 [9.99145901e-01]
 [9.93196658e-01]
 [2.49034769e-03]
 [9.94494198e-01]
 [7.58817780e-03]
 [1.25102286e-02]
 [1.90662557e-02]
 [3.57542771e-03]
 [9.96445342e-01]
 [3.33352957e-03]
 [9.98086690e-01]
 [1.47886131e-02]
 [9.96463431e-01]
 [5.85599136e-03]
 [9.98497064e-01]
 [9.69043803e-01]
 [9.98981143e-01]
 [9.96916694e-01]
 [9.93391444e-01]
 [5.19085219e-03]
 [1.13729062e-02]
 [4.68153342e-03]
 [4.89949711e-03]
 [6.38346727e-03]
 [9.93145455e-01]
 [9.99042843e-01]
 [9.41521992e-03]
 [1.55444956e-02]
 [9.72475555e-01]
 [9.97987119e-01]
 [9.89551483e-01]
 [2.13616874e-02]
 [9.93226722e-01]

## Write predict function for new data
- The function should use beta calculated in gradient descent and Xtest to return both prediciton and predicted probability.

In [16]:
# use trained beta to predict new data
def predict(beta, X):
    """
    Args:
        X (numpy array): feature to predict
        
    Returns:
        y (numpy array): predict results (0 or 1)
    """
    m = X.shape[0]
    X = np.hstack((np.ones((m, 1)), X))    # add constant column to X for intercept
    pred_proba = sigmoid(np.dot(X, beta)).ravel()
    print(pred_proba.shape)
    pred = np.array([round(x) for x in pred_proba])
    return pred, pred_proba

In [17]:
# find accuracy of model on test data
from sklearn.metrics import accuracy_score
ypred, ypred_proba = predict(beta_final, Xtest)
accuracy_score(ytest, ypred)

(25,)


1.0

In [18]:
# see if the predicted probabilities look normal 
ypred_proba

array([9.95799570e-01, 9.98890025e-01, 9.97852823e-01, 4.20178672e-03,
       9.97741322e-01, 9.98418422e-01, 5.11400991e-02, 9.98265663e-01,
       9.93678890e-01, 9.96196031e-01, 7.46767781e-03, 9.74623091e-01,
       5.47415895e-04, 6.81204464e-03, 9.95258850e-01, 6.50663366e-03,
       3.03637496e-03, 9.99510463e-01, 1.12998599e-02, 1.20260491e-03,
       4.47302413e-03, 2.15560638e-02, 1.03092611e-02, 4.60555138e-03,
       9.78354426e-01])

The accuracy is 1.0 and the prediced probabilities look normal, great! Note that this implementation is only a simple version, it may not work well in more complex dataset. More details like <b>regularization</b> and <b>weighted cost function</b> (for imbalanced data) can be added to advance the model. Also, data preprocessing like <b>feature scaling</b> may be required.